In [8]:
import requests
import json
import csv
import pandas as pd
from itertools import chain
from collections import Counter
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [9]:
file_path = 'movies_data_encoded.csv'  # Replace with the actual file name or path
movie_data_df = pd.read_csv(file_path)

pd.set_option('display.max_colwidth', 13)
display(movie_data_df)

,title,year,original_language,runtime_minutes,TMDB_popularity,vote_count,vote_rating,budget_($M),budget_bracket,revenue_($M),...,keywords_terrorist,keywords_texas,keywords_time_travel,keywords_torture,keywords_undercover,keywords_vampire,keywords_wedding,keywords_witch,keywords_woman_director,keywords_world_war_ii
0,Four Rooms,1995-12-09,en,98,21.376,2357,5.758,4.00000,low,4.257354,...,0,0,0,0,0,0,0,1,1,0
1,Judgment ...,1993-10-15,en,109,7.675,284,6.556,21.00000,medium,12.136938,...,0,0,0,0,0,0,0,0,0,0
2,Star Wars,1977-05-25,en,121,120.016,18563,8.208,11.00000,medium,775.398007,...,0,0,0,0,0,0,0,0,0,0
3,Finding Nemo,2003-05-30,en,100,74.931,17613,7.823,94.00000,high,940.335536,...,0,0,0,0,0,0,0,0,0,0
4,Forrest Gump,1994-06-23,en,142,52.131,24570,8.481,55.00000,high,677.387716,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,Jackass P...,2013-10-22,en,92,14.690,1407,6.104,15.00000,medium,151.831537,...,0,0,0,0,0,0,0,0,0,0
6937,Forbidden...,2014-01-30,ru,128,9.949,252,5.900,26.00000,medium,34.592118,...,0,0,0,0,0,0,0,0,0,0
6938,Batman v ...,2016-03-23,en,152,65.151,16721,5.900,250.00000,high,873.637528,...,0,0,0,0,0,0,0,0,0,0
6939,Zombie Br...,1986-05-22,en,92,1.129,7,2.500,0.00001,low,0.000015,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# ADDS MONTHS AND SEASONS 

# Convert the year column to datetime if not already done so
movie_data_df['year'] = pd.to_datetime(movie_data_df['year'])

# Generate the month names
month_names = movie_data_df['year'].dt.month_name()

# Define a function that maps a month to a season
def month_to_season(month):
    if month in ['March', 'April', 'May']:
        return 'Spring'
    elif month in ['June', 'July', 'August']:
        return 'Summer'
    elif month in ['September', 'October', 'November']:
        return 'Autumn'
    else:
        return 'Winter'

# Apply the function to the month names to get the seasons
seasons = month_names.apply(month_to_season)

# Convert the year column back to just the year
movie_data_df['year'] = movie_data_df['year'].dt.year

# Get the position of the year column
idx = movie_data_df.columns.get_loc('year')

# Insert the new columns
movie_data_df.insert(idx+1, 'month', month_names)
movie_data_df.insert(idx+2, 'season', seasons)


In [4]:
display(movie_data_df)

,title,year,month,season,original_language,runtime_minutes,TMDB_popularity,vote_count,vote_rating,budget_($M),...,keywords_terrorist,keywords_texas,keywords_time_travel,keywords_torture,keywords_undercover,keywords_vampire,keywords_wedding,keywords_witch,keywords_woman_director,keywords_world_war_ii
0,Four Rooms,1970.0,January,Winter,en,98,21.376,2357,5.758,4.00000,...,0,0,0,0,0,0,0,1,1,0
1,Judgment ...,1970.0,January,Winter,en,109,7.675,284,6.556,21.00000,...,0,0,0,0,0,0,0,0,0,0
2,Star Wars,1970.0,January,Winter,en,121,120.016,18563,8.208,11.00000,...,0,0,0,0,0,0,0,0,0,0
3,Finding Nemo,1970.0,January,Winter,en,100,74.931,17613,7.823,94.00000,...,0,0,0,0,0,0,0,0,0,0
4,Forrest Gump,1970.0,January,Winter,en,142,52.131,24570,8.481,55.00000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,Jackass P...,1970.0,January,Winter,en,92,14.690,1407,6.104,15.00000,...,0,0,0,0,0,0,0,0,0,0
6937,Forbidden...,1970.0,January,Winter,ru,128,9.949,252,5.900,26.00000,...,0,0,0,0,0,0,0,0,0,0
6938,Batman v ...,1970.0,January,Winter,en,152,65.151,16721,5.900,250.00000,...,0,0,0,0,0,0,0,0,0,0
6939,Zombie Br...,1970.0,January,Winter,en,92,1.129,7,2.500,0.00001,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Save the DataFrame to a CSV file
movie_data_df.to_csv('movies_data_finalised.csv', index=False, encoding='utf-8')
